In [183]:
import csv
from typing import List,Dict
import copy
import json
from collections import OrderedDict
from torch.utils.data import  Dataset

In [6]:
def read_csv(path: str) -> List[Dict[str, str]]:
    """Reads a csv file, and returns the content inside a list of dictionaries.
    Args:
    path: The path to the csv file.
    Returns:
    A list of dictionaries. Each row in the csv file will be a list entry. The
    dictionary is keyed by the column names.
    """
    with open(path, 'r') as f:
        return list(csv.DictReader(f))

In [92]:
mapping_table = read_csv('./inaturalist-user-120k/federated_train_user_120k.csv')

In [8]:
import enum

In [9]:
class INaturalistSplit(enum.Enum):
    """The different split for the iNaturalist dataset."""
    USER_120K = enum.auto()
    GEO_100 = enum.auto()
    GEO_300 = enum.auto()
    GEO_1K = enum.auto()
    GEO_3K = enum.auto()
    GEO_10K = enum.auto()
    GEO_30K = enum.auto()

    def __repr__(self):
        return '<%s.%s>' % (self.__class__.__name__, self.name)

In [10]:
repr(INaturalistSplit)

"<enum 'INaturalistSplit'>"

In [12]:
print(repr(INaturalistSplit.USER_120K))

<INaturalistSplit.USER_120K>


In [13]:
for name in INaturalistSplit:
    print(name)

INaturalistSplit.USER_120K
INaturalistSplit.GEO_100
INaturalistSplit.GEO_300
INaturalistSplit.GEO_1K
INaturalistSplit.GEO_3K
INaturalistSplit.GEO_10K
INaturalistSplit.GEO_30K


In [14]:
split = INaturalistSplit.USER_120K

In [15]:
print(type(split))

<enum 'INaturalistSplit'>


In [16]:
split.value

1

In [17]:
user_id_col = split.name.lower()

In [18]:
print(user_id_col)

user_120k


In [19]:
expected_cols = [user_id_col, 'image_id', 'class']

In [93]:
mapping_table[0].keys()

odict_keys(['user_id', 'image_id', 'class', 'label'])

In [23]:
mapping_table[1].keys()

odict_keys(['user_id', 'image_id', 'class', 'label'])

In [26]:
print(len(mapping_table))

120300


In [27]:
print(mapping_table[0])

OrderedDict([('user_id', 'J__Maughn'), ('image_id', 'e619826038358b817d926d869a0bbcf1'), ('class', '774'), ('label', '49463')])


In [62]:
user_id_set = set()

In [63]:
for row in mapping_table:
    user_id_set.add(row['user_id'])

In [49]:
print(len(user_id_set))

9275


In [50]:
print(user_id_set)

{'Nicholas_Eaton', 'Adam_Heathcote', 'scollins', 'merson', 'jeffbenn', 'plawn', 'rhyax', 'jahl', 'mmbrevig', 'jacob_miller', 'aynhart', 'na_ma778899', 'diego27', 'br1dgy', 'batgirl', 'Paul_Wheeler', 'lizarrrd55', 'mkashuba', 'lipstick6', 'parkecology', 'senchack', 'Joe_Walewski', 'wadewilsonisreal', 'deasunja', 'Karina_Duarte', 'brittanycushman', 'mpi', 'tony_wills', 'nbkdy9k', 'hristau', 'majmal1', 'cassandra001', 'E_Wright', 'lakekoshare', 'Sharie_Wiseman', 'rbossuk', 'waterrocks', 'bmarti66', 'David_Hutchinson', 'Mary_Morrow', 'meallen12', 'kiranp', 'Daniel_Joseph_Le_Saint', 'mfontenot22', 'spradoirwin', 'David_Gomez', 'dacapo3x', 'detectives_de_la_onda', 'abhcynical', 'Jim_Tate', 'Robbie_Gallant', 'divemaster', 'Mike_Thomas', 'lkaguilar', 'jessecavalencia', 'Maria_Concepcion_Perez_Recoder', 'University_of_Arizona_Sky_School', 'margievaldez', 'richardv', 'Bryn_Crutchfield', 'enzedfred', 'onesoaringspirit', 'woodleigh', 'jschweg', 'Greg_Caspers', 'saem', 'Tom_Devitt', 'Jose_Perez_Avi

In [54]:
print('100460587416170000000' in user_id_set)

False


In [57]:
print('John_BArkla' in user_id_set)

False


In [68]:
user_id_dict = {}

In [69]:
for idx,id in enumerate(user_id_set):
    user_id_dict[idx] = id

In [70]:
print(user_id_dict[0])

Nicholas_Eaton


In [71]:
user_id2idx_dict = {}

In [72]:
for idx,id in enumerate(user_id_set):
    user_id2idx_dict[id] = idx

In [73]:
print(user_id2idx_dict['john_barkla'])

9189


In [74]:
print(user_id2idx_dict['John_Barkla'])

6908


In [169]:
3 not in [1,2,4,5]

True

In [175]:

client_data = OrderedDict()
for idx,id in enumerate(user_id_set):
    client_data[id] = OrderedDict()
    client_data[id]['client_idx'] = idx
    client_data[id]['client_data'] = []

this_mapping_table = copy.deepcopy(mapping_table)

client_idx_data = OrderedDict()
for i in range(len(user_id_set)):
    client_idx_data[i] = []
count = 0
for row in this_mapping_table:
    
    this_user_id = row.pop('user_id')
    print(this_user_id)
    row['data_idx'] = count
    
    client_data[this_user_id]['client_data'].append(row)
    this_client_idx = client_data[this_user_id]['client_idx']
    client_idx_data[this_client_idx].append(count)
    count+=1
    

J__Maughn
Richard_Smart
Damon_Tighe
Liam_O_Brien
Eric_Jacob
Jason_Reiber
Jose_Francisco_Gomez_Rodriguez
Jonathan_Carpenter
Robin_Agarwal
vasey2113
James_Agyei_Ohemeng
Ken_ichi_Ueda
katelundquist
Andrey_Danilin
J__Maughn
Natalie_Weber
susanpow
Dominic
Nicolas_Olejnik
BJ_Stacey
Zac_Cota
colinmorita
Linda_Jo_Conn
avocat
mattiabuttarelli
Robin_Agarwal
Greg_Lasley
dlbowls
Zac_Cota
Greg_Lasley
l_sanford
catharus
fsi1212
Robert_J___Bob__Nuelle__Jr__AICEZS
greglasley
emilyd47
Donna_Pomeroy
Sam_Kieschnick
Christopher_E__Smith
Megan_Sebasky
rnunez09
Xavier_Ma_Ma
senmichaelwang
J__Maughn
Erika_Mitchell
sciencebabe
Jim_West
Ken_ichi_Ueda
Jakob_Fahr
connorj
Marisa_or_Robin_Agarwal
salmonskyview
womanofearth
Andy
catchang
Susan_Elliott
Linda_Jo_Conn
danielletran
J__Maughn
Jennifer_Rycenga
Jon_Sullivan
Jason_Whittle
gyrrlfalcon
Laura
Larry_Thornton
charley
Javier_Alejandro
isisunit
kylejones
avocat
sidesplotch
Cassandra_Cullin
Greg_Lasley
dfwuw
Francisco_Farriols_Sarabia
Mark_Rosenstein
Susan_Elliott

Robin_Agarwal
Laura
116916927065934112165
Ken_ichi_Ueda
Andrew
Michael_D_Fox
mc_california
Mickey_Long
Paul_Norwood
Robert_D_Stevenson
Jennifer_Rycenga
beschwar
dwest
marabethg
Alex_Shepard
Liam_O_Brien
beschwar
icosahedron
avoznitza
roby
Donna_Pomeroy
Daniel_Carter
116916927065934112165
Carlos_D__Ross
lonnyholmes
Erika_Mitchell
Tom_Norton
Cory__Chia__Chiappone
J__Maughn
Charlie_Hohn
johnmarrow
sidesplotch
Linda_Jo_Conn
Chris_Hyde
Mike_Leveille
Greg_Lasley
Steven_Mlodinow
donbelldonbell
centratex
James_Bailey
John_Barkla
Alison_Young
dpom
Scott_Buckel
sidesplotch
116916927065934112165
gyrrlfalcon
krosenthal
Tonya_Yoder
Andreas_Rockstein
lindarasmussen
olivernaturepark
Cara
Joe_Walewski
Cooper_Wyatt
Kurtis_Himmler
arnel
Cheryl_Harleston
ditchontologist
gshugart
vulpita_fotograf
Todd_Folsom
Jon_Sullivan
Jon_Sullivan
nickguerrero
owenclarkin
roy_pilcher
cdmar10
Sam_Kieschnick
tkoffel
Nevin_Durish
Kim__Hyun_tae
Jean_Lou_Justine
Janet_Gingold
brandonint
Leon_Perrie
Carolyn_Gritzmaker
tbluei

Susan_Miller
Andreas_Rockstein
Kim__Hyun_tae
Kailah_Andersen
James_Mickley
BJ_Stacey
Greg_Lasley
BJ_Stacey
Marisa_or_Robin_Agarwal
Cullen_Hanks
Jessica_Gardella
Ken_ichi_Ueda
Kathy_Richardson
117625618697965837369
krosenthal
Sam_Kieschnick
dshell
timharris
Kelly_Fuerstenberg
Don_Loarie
Carolyn_Gritzmaker
Kim__Hyun_tae
Erick_Noe_Tapia_Banda
eliza_browning
Mike_troll_Dame
Dominic
Don_Loarie
sciencebabe
Melissa_McMasters
fyn_kynd
anewman
J__Maughn
Don_Loarie
lunagato51
Sarah_Carline
Ken_Potter
msr
Andrew
Damon_Tighe
Robin_Agarwal
iveirizarry
lukecmurphy
Jesse_Rorabaugh
Chiara_Catalano
Robin_Agarwal
Greg_Lasley
Marion_Zoller
Matthew_O_Donnell
Laura_Clark
Alyssa_Berger
Liam_O_Brien
sea_kangaroo
BJ_Stacey
Cory__Chia__Chiappone
Michael_Sager
Greenlaw_Klein
Emanuel_Smedbol
greglasley
sarahcarline
Alison_Young
Mark_Rosenstein
Jeremy_Goldsmith
smballou
Laura
seahunt
Cheryl_Harleston
molanic
kara_oliver
Donna_Mason_Spier
Rafael_Paredes_Montesinos
leafminer
luckypen
Erin_Faulkner
Jenny_Smith
katha

Erika_Leydig
Kent_McFarland
Eric_Hardy
Donna_Pomeroy
Emily
BJ_Stacey
BJ_Stacey
aarongunnar
AJ_Johnson
catchang
Judith_Lopez_Sikora
elizrose
Alysa
Javier_Alejandro
Noel_Pennington
Robin_Agarwal
Cory__Chia__Chiappone
janja
kevinhintsa
Allen_Bartell
116916927065934112165
jimk
kevinhintsa
srall
Marv_Elliott
Chuck_Sexton
Steven_B__Biggers
Martin_Grimm
owenclarkin
Sam_Kieschnick
Alfonso_Gutierrez_Aldana
Greg_Lasley
avocat
Stevie_Kennedy_Gold
Bob_O_Kennon
aarenas
Ian_Armitage
Mickey_Long
Cullen_Hanks
Sam_Kieschnick
jociebee
Bob_Hislop
Melissa_Brown
Rolling_Meadows_Farm
amarsh
thumbsonthefeet
loarie
Cecilie_Svenningsen
kueda
sunnetchan
Nick_Block
Annika_Lindqvist
Jonathan_Carpenter
xizitro33
caseytunia
Andrew_Simon
BJ_Stacey
Donna_Pomeroy
J__Maughn
Killarney_Provincial_Park
cmancivalano
anis
lkairy
Donna_Pomeroy
112692329998402018828
Ed_Yerke_Robins
NHM_Nature
Juan_Felipe_Restrepo_Mesa
anewman
Jessica
krosenthal
roypilcher
Linda_Jo
sea_kangaroo
terrydad2
Bob_O_Kennon
cherylkershaw
reddad
Scott

jokurtz
Charlie_Hohn
David_J_Barton
ewzuber
alicia_penney
paloma
carianne
Chris_Evers
paloma
Bob_Heitzman
ccunningham
anro
Jesse_Rorabaugh
pohanginapete
sea_kangaroo
Greg_Lasley
outdoorsie
ctwothree
James_Bailey
bbi2
Manuel_Becerril_Gonzalez
Fluff_Berger
natalie
Carolyn_Gritzmaker
jsackel7
Lee_Castiglione
Reserva_de_la_Biosfera_Mariposa_Monarca
nollster
ellen_hildebrandt
anewman
Judith_Lopez_Sikora
Marisa_Rafter
Sam_Kieschnick
Sterling_Sheehy
randyshonkwiler
Diana_Terry_Hibbitts
Beate___Heinz_Beyerlein
Donna_Pomeroy
Leon_Perrie
goulda
deandsusan
Erik_Attaway
Kim__Hyun_tae
Marisa_or_Robin_Agarwal
Erika_Mitchell
Lameace_Hussain
Andy
Charlie_Hohn
Connor
Ashton_Tassin
judywelna
Rob_Curtis
thehowlingfantod
Rebecca_Johnson
Liam_O_Brien
srall
Rogelio_Martinez
Mike_Leveille
Francisco_Valdes_Perezgasga
Liam_O_Brien
meng_zhao
Scott_Buckel
abr
taliamulder
Roberto_R__Calderon
Nina
Greg_Lasley
Milo_Pyne
Sam_Kieschnick
Sam_Kieschnick
Mr__Wolf
anewman
Corey_Lange
aliciaa34
Liam_O_Brien
lst
mariam_ben

...

John_Bray
Michael_D_Fox
Melanie_Schuchart
Cullen_Hanks
tnewman
Dave_Behm
kevinhintsa
Greg_Lasley
Erick_Noe_Tapia_Banda
Linda_Jo
Judith_Lopez_Sikora
James_Bailey
johnbotany
icosahedron
sea_kangaroo
111995886493235165854
Alison_Young
Mike_Leveille
luckypen
mrpete
ellen_hildebrandt
kkvinson
paloma
mbarr
J__Maughn
Wendy_Anthony
parkecology
Mark_Rosenstein
victoriousness
J__Maughn
Bob_Heitzman
Matthew_Salkiewicz
James_Mickley
jeandotson
NHM_Citizen_Science_Program
Matthew_Herron
Donna_Pomeroy
ellen_hildebrandt
naturenut
biosam
andriy
jorgetrevino
Robert_L_Curtis
rfedders
griff
madalyn6690
John_Boback
sidesplotch
roy_pilcher
Ken_ichi_Ueda
replicon
ellen_hildebrandt
Jason_Singhurst
ninamarieb
Ethan_Gold
sharonn
Marisa_or_Robin_Agarwal
Mark_Rosenstein
Jenny_Smith
Corey_Lange
skiman1016
Robin_Agarwal
Lewnanny_Richardson
Carolyn_Gritzmaker
Bryan
Bruno_Durand
J__Maughn
Jesus_Vadillo
avocat
Linda_Jo
bradyjross
Alex_Hall
112692329998402018828
Greg_Lasley
Gilberto_Ponce_Tejeda
Marion_Zoller
ashley_m

Ron_Matsumoto
Jon_Sullivan
kylejones
griff
evening
Linda_Jo_Conn
Marion_Zoller
nolo_88
whitleygilbert
Bob_O_Kennon
quinn
garyg
texasblonde
blove
skystevens
gyrrlfalcon
brookesmith
BJ_Stacey
Linda_Jo_Conn
Javi_Gonzalez
gcira
sea_kangaroo
Ben_Hutchins
slsfirefight
Sam_Kieschnick
jwochos
gyrrlfalcon
bvant3
anewman
Jennifer_Rycenga
danielletran
Emily_Dove_Barton
J__Maughn
Dino_Biancolini
Cullen_Hanks
michael_hodge
Paul_Lewis
Dominic
lumo
Joe_Potter_Butler
javaro
kueda
skystevens
Laura
Charlotte_Bill
chamaeleo
Don_Loarie
Giacomo_Gola
Beate___Heinz_Beyerlein
Larry_Snyder
anewman
lizlovesnature
Greg_Lasley
Christopher_Tracey
Alison_Young
Erika_Mitchell
dwest
Fernando_Cecor
Cheryl_Harleston
Gonzalo_Zepeda_Martinez
Linda_Jo_Conn
Donna_Pomeroy
terrydad2
Susan_Elliott
greglasley
Judith_Lopez_Sikora
Kent_McFarland
zealandia2
John_Beetham
sea_kangaroo
James_Bailey
amadorios
Alison_Young
Marisa_or_Robin_Agarwal
texastrooper
ellen_hildebrandt
Laura_Habel
nanorca13
James_Bailey
Paul_Fenwick
Greg_Lasle

omartorrest
Marisa_or_Robin_Agarwal
Jon_Sullivan
Matthew_Salkiewicz
Jon_Sullivan
Marv_Elliott
leslie_flint
Jennifer_Rycenga
Martin_Grimm
Marty_Calabrese
Chuo_Chuo_Long_
Marv_Elliott
Sam_Kieschnick
Rebecca
glhans
Marv_Elliott
Robin_Agarwal
Brad_Grimm
Lindsey_K__Wise
Francisco_Farriols_Sarabia
Linda_Jo
Ken_ichi_Ueda
Martin_Grimm
loarie
Mandy_Hague
Jon_Sullivan
krosenthal
susanelliott
Ken_ichi_Ueda
Evan_Raskin
Matt_Flower
ellen_hildebrandt
Erin_Taylor
yaoshawn
Rebecca
Linda_Jo_Conn
Jim_Varnum
jesscherry
Mark_Nenadov
Lynn_Michael
Michelle_Hellstern
Donna_Pomeroy
filippoceccolini
dlbowls
roby
lincoln
abr
andrea_borelli
Judith_Lopez_Sikora
jokurtz
icosahedron
Cesar_Pollo
sciencebabe
cwwood
Cathy_Bell
jsackel7
Francisco_Farriols_Sarabia
Nathalie_Martin
Marisa_Agarwal
aarongunnar
jessmorian
dwest
roy_pilcher
Mark_Rosenstein
ProtoRealist
Sam_Kieschnick
Dominic
Cheryl_Harleston
Gavin_Kingsley
Michael_Kielb
John_B
Zac_Cota
Nate_Harvey
Donna_Pomeroy
Remington_Jackson
wildmarin
beschwar
sea_kangaro

Ted_Pick_Jr
jokurtz
Luis_Guillermo
Suzanne_Cadwell
Marv_Elliott
luranga
dhalter
aaron_fool
idahosundevil
Erika_Mitchell
John_Bray
Kaitlyn_Forks
nudibranchmom
Wendy_Herniman
Liam_O_Brien
Mark_Rosenstein
Greg_Lasley
Lauren_Nichols
srall
Charlie_Justus
lenaz
Cory__Chia__Chiappone
Greg_Lasley
mtsuleahc
jsoukhome
martina23
Scott_Buckel
Annika_Lindqvist
Belinda_Lo
Kenneth_Bader
loarie
Richard_Barnes
Zac_Cota
quinteronaturalista
BJ_Stacey
purplesagela
john_barkla
Scott_Buckel
anis
J__N__Stuart
Jennifer_Rycenga
Sam_Kieschnick
miguelbernal
ellen_hildebrandt
Chuck_Sexton
halisteinmann
KATERINE_ENDER_CORDOVA
115501263447894410250
Jesse_Rorabaugh
rokzsoul
tangatawhenua
lenaz
Logan_Ediger
ghanwa
Romey_Swanson
Linda_Jo_Conn
Luca_Maurino
Erick_Noe_Tapia_Banda
lisa_bennett
angelicabecerrat
rpgleason
brendahoffman324
Kelly_Fuerstenberg
James_Maughn
andrewdavies9000
avocat
petekane47
Kari_Pihlaviita
lisa_bennett
Edi
Ken_ichi_Ueda
Zac_Cota
116916927065934112165
Nancy
kathelma
Sam_Kieschnick
mduncan
Sam_K

James_Bailey
Marion_Zoller
Javi_Gonzalez
nhmordenana
Liam_O_Brien
Michael_Kielb
Martin_Grimm
imdezus
Robin_Agarwal
tiyumq
icosahedron
Cullen_Hanks
mbalame99
Howard_Friedman
Steven_B__Biggers
anro
luke_pearson
parkecology
Bruce_J__Mohn
Matt_Muir
scottstolsenberg
taraoakes1
kemmons
Roger_Shaw
Donna_Pomeroy
angelicabecerrat
Gena_Bentall
Chris_Mallory
Jennifer_Rycenga
Jon_Sullivan
dwest
Cullen_Hanks
liapun312
Jennifer_Rycenga
Ed_Wilson
altamiranohg
Dominic
Fernando_Ortiz
Marv_Elliott
roby
hummer56
pocketmouse
Joe_Walewski
marikcl
Greg_Seitz
owenclarkin
116916927065934112165
gcjohn
ohmygodjuju
epitree
Sarah_Carline
Ken_ichi_Ueda
michaelstringer
Javier_Alejandro
Mike_Leveille
casey_rice
Dominic
BJ_Stacey
Erika_Mitchell
Damon_Tighe
aliciaa34
112692329998402018828
biddles
faerthen
Donna_Pomeroy
116916927065934112165
Tyler_Christensen
Jenny_Smith
Liam_O_Brien
Mark_Lehman
_vmr_vynr
Suzanne_Cadwell
filippoceccolini
vfazio
tienglander
e_aus_kanada
J__Maughn
BJ_Stacey
Leanne
Matt_Muir
Frits_Schoute

In [176]:
client_data['Adam_Heathcote']


OrderedDict([('client_idx', 1),
             ('client_data',
              [OrderedDict([('image_id', 'c2cfd0484188da465b09e0ff962de74d'),
                            ('class', '1001'),
                            ('label', '9346'),
                            ('data_idx', 4707)]),
               OrderedDict([('image_id', '7db1ea38b4d361fcaf37a477acc80cfb'),
                            ('class', '1110'),
                            ('label', '57458'),
                            ('data_idx', 15026)]),
               OrderedDict([('image_id', '1adc627e90a6ed9349074c0bc528d6b7'),
                            ('class', '1041'),
                            ('label', '6433'),
                            ('data_idx', 16102)]),
               OrderedDict([('image_id', 'fdef4f285a898ccc73618d6c76f872a5'),
                            ('class', '982'),
                            ('label', '118078'),
                            ('data_idx', 20355)]),
               OrderedDict([('image_id', 'da47

In [180]:
print(client_idx_data[1])

[4707, 15026, 16102, 20355, 26947, 31606, 31645, 31719, 35236, 50474, 52116, 53646, 53788, 57782, 58900, 61756, 73053, 75472, 81102, 82197, 82974, 85952, 87984, 97589, 98691, 99735, 101126, 103636, 110651, 116355]


In [179]:

filepath = "./iNaturalist_client_idx.txt"
with open(filepath, 'w') as f:
    print(dict(client_idx_data), file=f)

In [156]:
mapping_table[0]

OrderedDict([('user_id', 'J__Maughn'),
             ('image_id', 'e619826038358b817d926d869a0bbcf1'),
             ('class', '774'),
             ('label', '49463')])

In [157]:
mapping_table[16102]

OrderedDict([('user_id', 'Adam_Heathcote'),
             ('image_id', '1adc627e90a6ed9349074c0bc528d6b7'),
             ('class', '1041'),
             ('label', '6433')])

In [ ]:
mapping_table[]

In [125]:
file_path = "./iNaturalist_client.json"
with open(file_path,'w') as outfile:
    json.dump(client_data,outfile, indent=4)

In [109]:
import copy
a={'1':1,'2':2}
b= copy.deepcopy(a)
z=b.pop('1')
print(z)
print(a)

1
{'1': 1, '2': 2}


KeyError: '1'

In [28]:
all(col in mapping_table[0].keys() for col in expected_cols)

False

In [29]:
expected_cols

['user_120k', 'image_id', 'class']

In [34]:
import os

In [31]:
def _generate_image_map(image_dir: str) -> Dict[str, str]:
    """Create an dictionary with key as image id, value as path to the image file.
    Args:
    image_dir: The directory containing all the images.
    Returns:
    The dictionary containing the image id to image file path mapping.
    """
    image_map = {}
    for root, _, files in os.walk(image_dir):
        for f in files:
            if f.endswith('.jpg'):
                image_id = f.rstrip('.jpg')
                image_path = os.path.join(root, f)
                image_map[image_id] = image_path
    return image_map

In [32]:
root = '/MD1400/jinkyu/train_val_images'
version = '2017'
download = False

In [36]:
image_map = _generate_image_map(root)

In [38]:
type(image_map)

dict

In [43]:
print(len(image_map.keys()))

675170


In [118]:
print(list(image_map.keys())[0])

5f42986e4de1713b67d8b1b58c49e7af


In [185]:
1+3

4

In [186]:
3%(-1)

0